In [1]:
import json
from scripts.parsers import parse_sequences

sequence = 'p7s1'

file_path = 'gait3d\\ListOfSequences.txt'
sequences = parse_sequences(file_path)
print(json.dumps(sequences[sequence], indent=4))

{
    "start_frame": 90,
    "number_of_frames": 120,
    "frame_offset": 0,
    "MoCap_data": true
}


In [2]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [3]:
import plotly.graph_objects as go
import numpy as np
from pyacclaim import loadASF, loadAMC

def create_skeleton_figure(asf, amc, start_frame=0, markers_color='blue', frames_per_sec=25):
    bones = asf["bones"]
    initial_frame = amc[start_frame]
    bones["root"].set_pose(initial_frame)
    
    x_coords, y_coords, z_coords = [], [], []
    for bone in bones.values():
        if bone.parent is not None:
            x_coords.extend([bone.global_coordinate[2], bone.parent.global_coordinate[2], None])
            y_coords.extend([bone.global_coordinate[0], bone.parent.global_coordinate[0], None])
            z_coords.extend([bone.global_coordinate[1], bone.parent.global_coordinate[1], None])

    fig = go.Figure(
        data=[go.Scatter3d(
            x=x_coords,
            y=y_coords,
            z=z_coords,
            mode='markers+lines',
            marker=dict(size=4, color=markers_color)
        )]
    )

    frames = []
    for i, pose in enumerate(amc[start_frame:]):
        bones["root"].set_pose(pose)
        
        x_coords, y_coords, z_coords = [], [], []
        for bone in bones.values():
            if bone.parent is not None:
                x_coords.extend([bone.global_coordinate[2], bone.parent.global_coordinate[2], None])
                y_coords.extend([bone.global_coordinate[0], bone.parent.global_coordinate[0], None])
                z_coords.extend([bone.global_coordinate[1], bone.parent.global_coordinate[1], None])
        
        frames.append(
            go.Frame(
                data=[go.Scatter3d(
                    x=x_coords,
                    y=y_coords,
                    z=z_coords,
                    mode='markers+lines',
                    marker=dict(size=4, color=markers_color)
                )],
                name=f'frame {i + start_frame}',
                layout=go.Layout(
                    annotations=[dict(
                        text=f'Frame {i + start_frame + 1}',
                        x=0.05, y=0.95, xref='paper', yref='paper',
                        showarrow=False, font=dict(size=16, color="black"))]
                )
            )
        )

    fig.frames = frames

    # Set layout for the figure
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[-50, 50]),
            yaxis=dict(range=[-175, 175]),
            zaxis=dict(range=[0, 100]),
            aspectmode='manual',
            aspectratio=dict(x=2, y=7, z=2)
        ),
        title='3D Skeleton Animation from ASF/AMC Files',
        width=800,
        height=600,
        annotations=[dict(
            text=f'Frame {start_frame + 1}',
            x=0.05, y=0.95, xref='paper', yref='paper',
            showarrow=False, font=dict(size=16, color="black"))],
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, dict(frame=dict(duration=1000 / frames_per_sec, redraw=True), fromcurrent=True)])]
        )]
    )

    fig.show()

# Example usage:
sequence = 'p7s1'
asf = loadASF(f"./gait3d/Sequences/{sequence}/MoCap/{sequence}.asf")
print(asf)
print(type(asf))
amc = loadAMC(asf, f"./gait3d/Sequences/{sequence}/MoCap/{sequence}.amc")
print(type(amc))
print(type(amc[0]))
print(amc[0])
print(type(amc[0]['root']))
start_frame = sequences[sequence]['start_frame'] * 4 #mocap in 100 fps
create_skeleton_figure(asf, amc, start_frame=start_frame, frames_per_sec=100)

[loadASF] Reading contents of ./gait3d/Sequences/p7s1/MoCap/p7s1.asf
... Version: 1.10
... Name: VICON
... Description: .ast/.asf automatically generated from VICON data using VICON BodyBuilder and BodyLanguage model FULLBODY5.MOD
... Units:
        Mass: 1.0
        Length: 0.1
        Angle: deg
[loadASF] Success.
{'version': '1.10', 'name': 'rhand', 'description': '.ast/.asf automatically generated from VICON data using VICON BodyBuilder and BodyLanguage model FULLBODY5.MOD', 'units': {'mass': 1.0, 'length': 0.1, 'angle': 'deg'}, 'bones': {'root': <pyacclaim.bone.Bone object at 0x0000022029EF7EB0>, 'lhipjoint': <pyacclaim.bone.Bone object at 0x0000022029EF7E50>, 'lfemur': <pyacclaim.bone.Bone object at 0x0000022029EF7EE0>, 'ltibia': <pyacclaim.bone.Bone object at 0x000002204837D210>, 'lfoot': <pyacclaim.bone.Bone object at 0x000002204837D360>, 'ltoes': <pyacclaim.bone.Bone object at 0x000002204837D390>, 'rhipjoint': <pyacclaim.bone.Bone object at 0x000002204837D150>, 'rfemur': <pyac

In [4]:
import numpy as np
import math

# attempts to do some corrections
def prepare_new_amc(amc, new_order=[0,1,2]):
    new_amc = []
    for frame in amc:
        new_frame = {}
        for bone, params in frame.items():
            if bone == 'root':
                new_frame[bone] = [params[0], params[1], params[2], 0, params[4],  0] # ignoring all rotations except Y gives best results  
                # new_frame[bone] = params[[0, 1, 2, new_order[0]+3, new_order[1]+3, new_order[2]+3]]
                # new_frame[bone] = params[[new_order[0], new_order[1], new_order[2], 3, 4, 5]]

                # new_frame[bone] = params
            else:
                # new_frame[bone] = params[new_order]  
                new_frame[bone] = [params[0], params[1], params[2]]
                #new_frame[bone] = params
        new_amc.append(new_frame)
        
    return new_amc

prepare_new_amc(amc[:2], new_order=[0,1,2])

[{'root': [np.float64(2.6301),
   np.float64(36.111399999999996),
   np.float64(-4.07093),
   0,
   np.float64(-1.414304870073329),
   0],
  'lowerback': [np.float64(-0.20520883213248528),
   np.float64(-0.038552577847302746),
   np.float64(0.01839681751357143)],
  'upperback': [np.float64(0.005098804876776234),
   np.float64(-0.055093935833079005),
   np.float64(0.012923569699072332)],
  'thorax': [np.float64(0.12032125330323709),
   np.float64(-0.026650479546252617),
   np.float64(-0.006384519123550377)],
  'lowerneck': [np.float64(0.02927772366927968),
   np.float64(-0.002115670665975006),
   np.float64(-0.0769143912073625)],
  'upperneck': [np.float64(-0.11127276832919769),
   np.float64(0.002077936647546889),
   np.float64(0.16441298259759424)],
  'head': [np.float64(-0.041616677882103985),
   np.float64(-0.002279836335417593),
   np.float64(0.06844343568280803)],
  'rshoulderjoint': [np.float64(-0.05656856451808902),
   np.float64(0.07509104573780402),
   np.float64(-0.8385818363

In [5]:
new_amc = prepare_new_amc(amc, new_order=[0, 1, 2])
# start_frame = 0 
start_frame = sequences[sequence]['start_frame'] * 4 #mocap in 100 fps
create_skeleton_figure(asf, new_amc, start_frame=start_frame, frames_per_sec=100)